### Ejercicio 1

A partir de la API de Pokemon, se pide generar un DataFrame como el de la imagen con las columnas: 

- **"height","id","order","weight","types"**

### Ejercicio 2

¿Has visto que dentro del DataFrame la columna "types" hay diccionarios? Se pide agregar, por cada pokemon, las columnas necesarias al DataFrame para albergar toda la información de "types".

![](df_pk.PNG)

In [12]:
import json
import requests
import pandas as pd

Lo primero de todo será definir la **url** de donde sacaremos la información y realizar la petición con la función `get` de la librería `requests`, pasándolo a su vez a `json`.

In [2]:
url = "https://pokeapi.co/api/v2/pokemon"
poke_json = requests.get(url).json()

Creamos una lista vacía donde contendremos todos los resultados obtenidos de la api, además de un diccionario vacío para guardar los nombre de pokemon y las **url** de donde luego sacaremos el resto de información 

In [3]:
pokemon = []
pokemon.extend(poke_json["results"])

A partir de los resultados creamos un dataframe.

In [4]:
df = pd.DataFrame(pokemon)
df

,name,url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/
3,charmander,https://pokeapi.co/api/v2/pokemon/4/
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/
5,charizard,https://pokeapi.co/api/v2/pokemon/6/
6,squirtle,https://pokeapi.co/api/v2/pokemon/7/
7,wartortle,https://pokeapi.co/api/v2/pokemon/8/
8,blastoise,https://pokeapi.co/api/v2/pokemon/9/
9,caterpie,https://pokeapi.co/api/v2/pokemon/10/


In [5]:
def get_info(url):
    lista_vacia = []
    peticion_api_url = requests.get(url).json()
    lista_keys = ["height","id","order","weight","types"]
    for key in lista_keys:
        lista_vacia.append(peticion_api_url[key])
    return pd.Series(lista_vacia)
        

In [6]:
df[["height","id","order","weight","types"]] = df['url'].apply(lambda x:get_info(x))

In [7]:
df.head()

,name,url,height,id,order,weight,types
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"[{'slot': 1, 'type': {'name': 'grass', 'url': ..."
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,6,4,5,85,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,11,5,6,190,"[{'slot': 1, 'type': {'name': 'fire', 'url': '..."


In [22]:
df['types'][0]

[{'slot': 1,
  'type': {'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'}},
 {'slot': 2,
  'type': {'name': 'poison', 'url': 'https://pokeapi.co/api/v2/type/4/'}}]

Para "sacar" el diccionario de la lista contenida en la columna `types` se deberá usar `pd.explode`.

- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html


In [8]:
first = df.explode("types")
first.head()

,name,url,height,id,order,weight,types
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,"{'slot': 2, 'type': {'name': 'poison', 'url': ..."
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,"{'slot': 1, 'type': {'name': 'grass', 'url': '..."


Este es el diccionario que hay que deshacer en este caso en 3 columnas, **slot**, **name** y **url**.

In [9]:
first['types'].values[0]

{'slot': 1,
 'type': {'name': 'grass', 'url': 'https://pokeapi.co/api/v2/type/12/'}}

Así no.

In [10]:
pd.DataFrame(first["types"].iloc[0])

,slot,type
name,1,grass
url,1,https://pokeapi.co/api/v2/type/12/


Así si.

In [ ]:
#0 sin usar json_normalize, A MANITA

In [31]:
def get_more_data(jsonito):
    return pd.Series([jsonito['slot'], jsonito['type']['name'], jsonito['type']['url']])

In [32]:
first[["slot", "type.name", "type.url"]] = first['types'].apply(lambda x: get_more_data(x))

In [33]:
first.drop(columns = "types", inplace = True)

In [34]:
first.head()

,name,url,height,id,order,weight,slot,type.name,type.url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,1,grass,https://pokeapi.co/api/v2/type/12/
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,2,poison,https://pokeapi.co/api/v2/type/4/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,1,grass,https://pokeapi.co/api/v2/type/12/
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,2,poison,https://pokeapi.co/api/v2/type/4/
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,1,grass,https://pokeapi.co/api/v2/type/12/


Con `json_normalize` podremos hacerlo fácilmente. Ya nunca más os costará deshacer un json/diccionario/lista dentro de un dataframe.

- Documentación oficial --> https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html
- Documentación no oficial --> https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe

In [36]:
#1
second = json_normalize(json.loads(first.to_json(orient="records")))
second.head()

<ipython-input-36-73ed5125d60a>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  second = json_normalize(json.loads(first.to_json(orient="records")))


,name,url,height,id,order,weight,slot,type.name,type.url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,1,grass,https://pokeapi.co/api/v2/type/12/
1,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,2,poison,https://pokeapi.co/api/v2/type/4/
2,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,1,grass,https://pokeapi.co/api/v2/type/12/
3,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,2,poison,https://pokeapi.co/api/v2/type/4/
4,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,1,grass,https://pokeapi.co/api/v2/type/12/


In [14]:
#2
third = pd.json_normalize(first["types"])
third.head()

,slot,type.name,type.url
0,1,grass,https://pokeapi.co/api/v2/type/12/
1,2,poison,https://pokeapi.co/api/v2/type/4/
2,1,grass,https://pokeapi.co/api/v2/type/12/
3,2,poison,https://pokeapi.co/api/v2/type/4/
4,1,grass,https://pokeapi.co/api/v2/type/12/


In [20]:
fourth = pd.concat([first.reset_index(drop = True), third.reset_index(drop = True)], axis=1)
fourth.drop(columns = ["types"], inplace = True)

In [21]:
fourth.head()

,name,url,height,id,order,weight,slot,type.name,type.url
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,1,grass,https://pokeapi.co/api/v2/type/12/
1,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,7,1,1,69,2,poison,https://pokeapi.co/api/v2/type/4/
2,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,1,grass,https://pokeapi.co/api/v2/type/12/
3,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,10,2,2,130,2,poison,https://pokeapi.co/api/v2/type/4/
4,venusaur,https://pokeapi.co/api/v2/pokemon/3/,20,3,3,1000,1,grass,https://pokeapi.co/api/v2/type/12/
